In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns

In [ ]:
train_df = pd.read_csv('content/train.csv')
weather_train_df = pd.read_csv('content/weather_train.csv')
building_meta_df = pd.read_csv('content/building_metadata.csv')

# Loading the data

## Merging the data

In [ ]:
train_df = train_df.merge(building_meta_df, on="building_id")
train_df = train_df.merge(weather_train_df, on=["site_id", "timestamp"])

In [ ]:
train_df.drop(columns=['Unnamed: 0','Unnamed: 0.1'], inplace=True)

In [ ]:
train_df

## reducing size for memory optim

In [ ]:
d_types = {'building_id': np.int16,
          'meter': np.int8,
          'site_id': np.int8,
           'floor':np.int8,
           'area': np.int8 ,
          'primary_use': 'category',
          'square_feet': np.int32,
          'year_built': np.float16,
          'floor_count': np.float16,
          'air_temperature': np.float32,
          'cloud_coverage': np.float16,
          'dew_temperature': np.float32,
          'precip_depth_1_hr': np.float16,
          'sea_level_pressure': np.float32,
          'wind_direction': np.float16,
          'wind_speed': np.float32}

for feature in d_types:
    train_df[feature] = train_df[feature].astype(d_types[feature])


## Changing timestamp type  into datetime

In [ ]:
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])

## Adding Month and day as new features

In [ ]:
train_df['day'] = train_df['timestamp'].dt.day
train_df['month'] = train_df['timestamp'].dt.month
train_df['hour'] = train_df['timestamp'].dt.hour

In [ ]:
train_df.head()

## creating categorical col out of th primary_use col

In [ ]:
 # categories are assumed to be the unique values of `values` (sorted, if possible, otherwise in the order in which they appear).
train_df['primary_use'] = pd.Categorical(train_df['primary_use'])
train_df['primary_use'] = train_df['primary_use'].cat.codes

In [ ]:
train_df.head()

percent of missing values

In [ ]:
100 * train_df.isnull().sum() / len(train_df)

## Dropping high missing values col

In [ ]:
del train_df['floor_count']
del train_df['year_built']

In [ ]:
train_df.columns

# Exploring Data for outliers

## keeping only meter 0 and 1

0: electricity
1: chilled water

In [ ]:
train_df.drop(index=train_df[ train_df['meter'] > 1].index, inplace=True )

In [ ]:
train_df.meter.unique()

In [ ]:
train_df.site_id.unique()

## Visualizing mean of energy per site

In [ ]:
fig, axes = plt.subplots(16,figsize=(15, 25), dpi=100)
for i in range(train_df['site_id'].nunique()):
    try:
        train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean().plot(ax=axes[i], alpha=0.8, label='By hour')
        axes[i].set_title('site_id {}'.format(i), fontsize=13);
        plt.subplots_adjust(hspace=0.6)
    except: pass

## Checking site 9

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(15, 10), dpi=100)
for i in list( train_df[(train_df['site_id'] == 9)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 9) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].legend();
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

### we can see clear outliers, lets check if it's a building issue or a certain type or buildings

In [ ]:
# train_df[(train_df['site_id'] == 9)]['primary_use'].unique()

In [ ]:
fig, axes = plt.subplots(6,1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 9)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 9) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### Clipping values

let's try clipping at 200

In [ ]:
clipped_values= train_df[(train_df['site_id'] == 9)].meter_reading.clip( upper=200, axis=0)
clipped_idx= train_df[(train_df['site_id'] == 9)].index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

let's visualize again

In [ ]:
fig, axes = plt.subplots(6,1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 9)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 9) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    #train_df[ (train_df['site_id'] == 9) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').var()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:red').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

well that's way better!

### so we decided keep site 9 and clip the values

## Checking site 13

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 13)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 13)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 13) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### so we decided to ditch site13's primary use 0

In [ ]:
train_df.drop(index=train_df[ (train_df['site_id'] == 9) & (train_df['primary_use'] == 0) ].index, inplace=True )

### Showing the effect of dropping the outlier

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 10), dpi=100)
for i in list( train_df[(train_df['site_id'] == 13)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 13) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

### clipping values

In [ ]:
condition= train_df[(train_df['site_id'] == 13) & (train_df.meter == 1)]
clipped_values= condition.meter_reading.clip(upper=1000, axis=0)
clipped_idx= condition.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

In [ ]:
condition= train_df[(train_df['site_id'] == 13) & (train_df.meter == 0) & (train_df.month.isin([3,4]) ) ]
clipped_values= condition.meter_reading.clip(upper=350, axis=0)
clipped_idx= condition.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

### Visualizing again

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 13)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 13)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 13) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Checking site 6

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 6)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 6)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 6) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 18), dpi=100)
for i in list( train_df[(train_df['site_id'] == 6)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 6) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].legend();
    axes[i].set_title('Meter: ' + str(i), fontsize=13);

### we see that primary_use 1 is the outlaier
it's clear that in one of the buildings the chilled water usage rose very high creatign a problem, but as it's hard to put our hands on the specific building we will just ditch the whole use

In [ ]:
train_df.drop(index=train_df[ (train_df['site_id'] == 6) & (train_df['primary_use'] == 1) ].index, inplace=True )

### clipping values

In [ ]:
condition= train_df[(train_df['site_id'] == 6) & (train_df.meter == 1) & (train_df.month == 3)]
clipped_values= condition.meter_reading.clip( upper=350, axis=0)
clipped_idx= condition.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

### Visualizing again after clipping

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 6)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 6)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 6) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Checking site 0

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 0)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 0)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 0) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 10), dpi=100)
for i in list( train_df[(train_df['site_id'] == 0)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 0) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

### We can see that for site 0 the problem is with electricity
 since it's for all of the usages, we wil lditch it, and then cut the percentage of the outlaier we can see in the chilled water meter

In [ ]:
train_df.drop(index=train_df[ (train_df['site_id'] == 0) & (train_df['meter'] == 0) ].index, inplace=True )

### Visualizing every primary_use now

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 0)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 0)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 0) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### Clipping outliers

In [ ]:
clipped_values= train_df[(train_df['site_id'] == 0) & ( train_df.primary_use==0) & ( train_df.month==5) ].meter_reading.clip( upper=5000, axis=0)
clipped_idx= train_df[(train_df['site_id'] == 0) & ( train_df.primary_use==0) & ( train_df.month==5) ].index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

In [ ]:
clipped_values= train_df[(train_df['site_id'] == 0) & ( train_df.primary_use==4) & ( train_df.month==7) ].meter_reading.clip( upper=5000, axis=0)
clipped_idx= train_df[(train_df['site_id'] == 0) & ( train_df.primary_use==4) & ( train_df.month==7) ].index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(15, 5), dpi=100)
train_df[(train_df['site_id'] == 0)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
axes.set_title('Meter: 1 ', fontsize=13);

Visualizing again for the primary_use after removing

In [ ]:
fig, axes = plt.subplots(len(train_df[(train_df['site_id'] == 0)]['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( train_df[(train_df['site_id'] == 0)]['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 0) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Checking site 9

In [ ]:
cond= train_df[(train_df['site_id'] == 9)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 9) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### Clipping

In [ ]:
condition_df= train_df[(train_df['site_id'] == 9) & ( train_df.primary_use==6)  ]
clipped_values= condition_df.meter_reading.clip( upper=40, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

In [ ]:
condition_df= train_df[(train_df['site_id'] == 9) & ( train_df.primary_use==12)  ]
clipped_values= condition_df.meter_reading.clip( upper=85, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

In [ ]:
condition_df= train_df[(train_df['site_id'] == 9) & ( train_df.primary_use==4)  ]
clipped_values= condition_df.meter_reading.clip( upper=30, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

In [ ]:
condition_df= train_df[(train_df['site_id'] == 9) & ( train_df.primary_use.isin([9,1]) ) ]
clipped_values= condition_df.meter_reading.clip( upper=100, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

### Visualizing after clipping

In [ ]:
cond= train_df[(train_df['site_id'] == 9)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 9) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Checking site 7

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 10), dpi=100)
for i in list( train_df[(train_df['site_id'] == 7)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 7) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

In [ ]:
cond= train_df[(train_df['site_id'] == 7)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 5), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 7) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes.set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### Ok seems fine and normal.

## Checking site 8

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(14, 5), dpi=100)
for i in list( train_df[(train_df['site_id'] == 8)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 8) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes.set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 8)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 8) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

so there are outliers higher than 150 so that will be our threshold

### Clipping

In [ ]:
condition_df= train_df[(train_df['site_id'] == 8) ]
clipped_values= condition_df.meter_reading.clip( upper=40, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

### Visualizing after clipping

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(14, 5), dpi=100)
for i in list( train_df[(train_df['site_id'] == 8)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 8) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes.set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 8)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 8) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Checking site 10

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 6), dpi=100)
for i in list( train_df[(train_df['site_id'] == 10)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 10) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 10)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 10) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### Clipping values

In [ ]:
condition_df= train_df[(train_df['site_id'] == 10)& (train_df.primary_use==0) ]
clipped_values= condition_df.meter_reading.clip( upper=150, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

# dropping primary use 7, 13 as it's range is just so suspecious
condition= train_df[(train_df['site_id'] == 10) & (train_df.primary_use.isin([7,13])) ]
train_df.drop(index=condition.index , inplace=True)

condition_df= train_df[(train_df['site_id'] == 10) & (train_df.primary_use==13)&(train_df.month==10) ]
clipped_values= condition_df.meter_reading.clip( upper=400, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

condition_df= train_df[(train_df['site_id'] == 10) & (train_df.primary_use==4) ]
clipped_values= condition_df.meter_reading.clip( upper=90, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

condition_df= train_df[(train_df['site_id'] == 10) & (train_df.primary_use==1) ]
clipped_values= condition_df.meter_reading.clip( upper=125, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

condition_df= train_df[(train_df['site_id'] == 10) & (train_df.meter==0)]
clipped_values= condition_df.meter_reading.clip( upper=80, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

### Visualizing after clipping

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 6), dpi=100)
for i in list( train_df[(train_df['site_id'] == 10)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 10) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 10)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 10) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Checking site 11

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 6), dpi=100)
for i in list( train_df[(train_df['site_id'] == 11)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 11) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 11)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 3), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 11) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes.set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### Dropping meter 1 as the missing data is skewing our graph

In [ ]:
cond= train_df[(train_df['site_id'] == 11) & (train_df.meter==1)]
train_df.drop(index=cond.index, inplace=True)

### Visualizing after removing data

In [ ]:
fig, axes = plt.subplots(1,1,figsize=(14, 6), dpi=100)
for i in list( train_df[(train_df['site_id'] == 11)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 11) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes.set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 11)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 3), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 11) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes.set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Lastly Checking site 14

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(14, 6), dpi=100)
for i in list( train_df[(train_df['site_id'] == 10)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 14) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 14)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 14) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### as we can see meter 1 is so huge that it creates an unbalanced data, so we will try visualizing without it


In [ ]:
# visualizing without meter 1

fig, axes = plt.subplots(2,1,figsize=(14, 6), dpi=100)
for i in list( train_df[(train_df['site_id'] == 10)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 14) & (train_df['meter'] == i)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[i], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes[i].set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 14)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 14) & (train_df['primary_use'] == i) & (train_df.meter ==0) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

### we clearly see that meter 1 has some problems with it, so we will drop it and clip some outliers

In [ ]:
condition_df= train_df[(train_df['site_id'] == 14)& (train_df.primary_use==1) ]
clipped_values= condition_df.meter_reading.clip( upper=230, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

# dropping primary use 7, 13 as it's range is just so suspecious
condition= train_df[(train_df['site_id'] == 14) & (train_df.meter==1) ]
train_df.drop(index=condition.index , inplace=True)

condition_df= train_df[(train_df['site_id'] == 14) & (train_df.primary_use==2)]
clipped_values= condition_df.meter_reading.clip( upper=250, axis=0)
clipped_idx= condition_df.index
train_df.loc[clipped_idx,'meter_reading']= clipped_values

### visualizing after removing outliers

In [ ]:
# visualizing without meter 1

fig, axes = plt.subplots(1,1,figsize=(14, 3), dpi=100)
for i in list( train_df[(train_df['site_id'] == 10)]['meter'].unique() ) :
    train_df[(train_df['site_id'] == 14)][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes, alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13);
    axes.set_title('Meter: ' + str(i), fontsize=13);
    plt.subplots_adjust(hspace=0.6)

cond= train_df[(train_df['site_id'] == 14)]
fig, axes = plt.subplots(len(cond['primary_use'].unique()),
                         1,figsize=(14, 18), dpi=100)
counter_var=0
for i in list( cond['primary_use'].unique() ) :
    train_df[ (train_df['site_id'] == 14) & (train_df['primary_use'] == i) ] [['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean()['meter_reading'].plot(ax=axes[counter_var], alpha=0.8, label='By hour', color='tab:blue').set_ylabel('Mean meter reading', fontsize=13)
    axes[counter_var].set_title('primary_use: ' + str(i), fontsize=13);
    counter_var+=1
    plt.subplots_adjust(hspace=0.6)

## Revisualizing all sites after preprocessing


In [ ]:
fig, axes = plt.subplots(16,figsize=(15, 25), dpi=100)
for i in range(train_df['site_id'].nunique()):
    try:
        train_df[train_df['site_id'] == i][['timestamp', 'meter_reading']].set_index('timestamp').resample('H').mean().plot(ax=axes[i], alpha=0.8, label='By hour')
        axes[i].set_title('site_id {}'.format(i), fontsize=13);
        plt.subplots_adjust(hspace=0.6)
    except: pass

# Saving the data

In [ ]:
train_df.to_csv('content/preprocessed_train.csv')